# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial
import xarray as xr

import utils
from parcels_utils import HFRGrid
from plot_utils import plot_particles, get_carree_axis, generate_simulation_plots

In [ ]:
files_root = Path("/Volumes/T7/Documents/Programs/scripps-cordc/parcels_westcoast")

# the data generated by ParticleFile
p_nc = utils.PARTICLE_NETCDF_DIR / "particle_plume_track.nc"
# particle data
p_ds = xr.open_dataset(p_nc)
p_ds.close()

# ocean current netcdf data the particles used
d_nc = files_root / utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_plume_interped.nc"
# ocean current data
d_info = HFRGrid(d_nc)
d_ds = d_info.xrds
fs = d_info.fieldset


In [ ]:
# the good ol' click coordinates on google maps
coastline_points = np.array([
    [32.527430, -117.123895],
    [32.534549, -117.123832],
    [32.549035, -117.125920],
    [32.556846, -117.130490],
    [32.560474, -117.131934],
    [32.562632, -117.132144],
    [32.564511, -117.132933],
    [32.575317, -117.132941],
    [32.576679, -117.133227],
    [32.593312, -117.133008],
    [32.616745, -117.136719],
    [32.640275, -117.146353],
    [32.662540, -117.160633],
    [32.679572, -117.180263],
    [32.687357, -117.198503],
    [32.683428, -117.223487]
])

lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline.mat", "latz0", "lonz0")
# lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline_1km.mat", "rlatz0", "rlonz0")
coastline_points = np.array([lats, lons]).T
coast_kdtree = scipy.spatial.KDTree(coastline_points)

tijuana_mouth = np.array([32.551967, -117.127208])

In [ ]:
def line_seg(x1, y1, x2, y2):
    return dict(
        x1=x1,
        y1=y1,
        x2=x2,
        y2=y2,
        dom=(x1, x2) if x1 <= x2 else (x2, x1),
        rng=(y1, y2) if y1 <= y2 else (y2, y1),
        # check for vertical line
        slope=(y1 - y2) / (x1 - x2) if x1 - x2 != 0 else np.nan
    )

def valid_point(x, y, line):
    in_dom = line["dom"][0] <= x <= line["dom"][1]
    in_range = line["rng"][0] <= y <= line["rng"][1]
    return in_dom and in_range

def intersection_info(x, y, line):
    """
    Returns:
        intersection x, intersection y
    """
    # vertical line
    if np.isnan(line["slope"]):
        return line["x1"], y
    if line["slope"] == 0:
        return x, line["y1"]
    norm_slope = -1 / line["slope"]
    slope_d = norm_slope - line["slope"]
    int_d = (line["slope"] * -line["x1"] + line["y1"]) - (norm_slope * -x + y)
    x_int = int_d / slope_d
    y_int = norm_slope * (x_int - x) + y
    return x_int, y_int

In [ ]:
# line segment information for the coastline
coast_segs = np.empty(len(coastline_points) - 1, dtype=dict)
for i in range(0, len(coastline_points) - 1):
    coast_segs[i] = line_seg(coastline_points[i][1], coastline_points[i][0], coastline_points[i + 1][1], coastline_points[i + 1][0])

In [ ]:
TIJUANA_MOUTH_DOMAIN = dict(
    S=32.53,
    N=32.564,
    W=-117.162,
    E=-117.105
)

# line segment representation of coastline
# ax = get_carree_axis(d_info.get_domain())
ax = get_carree_axis(TIJUANA_MOUTH_DOMAIN)
plt.scatter(coastline_points.T[1], coastline_points.T[0], s=2)
# plt.plot(coastline_points.T[1], coastline_points.T[0])

### distance to the coastline

get particles' closest distance to shore at every position and time saved in the particle data

In [ ]:
p_lats = p_ds["lat"].values
p_lons = p_ds["lon"].values
p_traj = p_ds["trajectory"].values

In [ ]:
collide_dist_thresh = 100  # meters
dists = np.full((p_ds.dims["traj"], p_ds.dims["obs"]), np.inf)
for i in range(dists.shape[0]):
    for j in range(dists.shape[1]):
        if np.isnan(p_traj[i, j]):
            dists[i, j:] = np.nan
            break
        # converting these to a python float makes it run twice as fast?
        lat = float(p_lats[i, j])
        lon = float(p_lons[i, j])
        least_dist = np.inf
        closest_idx = coast_kdtree.query([lat, lon])[1]
        if closest_idx == 0:
            seg_check = [coast_segs[closest_idx]]
        elif closest_idx == len(coastline_points) - 1:
            seg_check = [coast_segs[closest_idx] - 1]
        else:
            seg_check = [coast_segs[closest_idx - 1], coast_segs[closest_idx]]
        for seg in seg_check:
            lon_int, lat_int = intersection_info(lon, lat, seg)
            if valid_point(lon_int, lat_int, seg):
                dist = utils.haversine(lat, lat_int, lon, lon_int)
                least_dist = dist if dist < least_dist else least_dist
                if least_dist <= collide_dist_thresh:
                    break
        # point is in a region not normal to any of the line segments
        # compare distances to defined points and find closest
        pnt = coastline_points[closest_idx]
        dist = utils.haversine(lat, pnt[0], lon, pnt[1])
        least_dist = dist if dist < least_dist else least_dist
        dists[i, j] = least_dist
        if least_dist <= collide_dist_thresh:
            break

In [ ]:
d_mask = utils.conv_to_dataarray(utils.generate_mask_none(d_ds["u"].values), d_ds["u"].isel(time=0))

In [ ]:
def time_to_str(time):
    """
    haha time go chop chop
    
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### check whether a particle collided with land

whether it's within the defined distance with the coastline (in meters)

OR

if the particle is in a coordinate without data and has 0 velocity

### some encountered problems

- particles are out of bounds even though I use coordinate values on the SAME FIELDSET during simulation
    - ??????????????? how

### TODO mask instead of for loop to check

In [ ]:
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls_coll = []
lon_ls_coll = []
times_sec = []
for p_iter in np.where((dists <= collide_dist_thresh).sum(axis=1))[0]:
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_ds["time"].values[p_iter, step]
        time_secs = p_ds["lifetime"].values[p_iter, step] + p_ds["spawntime"].values[p_iter, step]
        lat = p_ds["lat"].values[p_iter, step]
        lon = p_ds["lon"].values[p_iter, step]
        times_sec.append(time_secs)
        # don't check if current data is nan anymore
        # check if it's close enough to the coastline
        near_coast = dists[p_iter, step] <= collide_dist_thresh
        if near_coast:
            collided_land[p_iter, step] = True
            lat_ls_coll.append(lat)
            lon_ls_coll.append(lon)
            print(f"particle {p_iter} near coast at time {time_to_str(time)}")
            print(f"    spawn time: {time_to_str(p_ds['time'].values[p_iter, 0])}")
            print(f"    spawn coord: ({p_ds['lat'].values[p_iter, 0]}, {p_ds['lon'].values[p_iter, 0]})")
            print(f"    lifetime: {p_ds['lifetime'].values[p_iter, step]}")
            # particle has hit coastline. we don't care about what happens after
            break

In [ ]:
# plot particles that have beached or something like that
domain = d_info.get_domain()
domain["W"] = -117.15
domain["E"] = -117.1
domain["N"] = 32.575
domain["S"] = 32.4
plot_particles(lat_ls_coll, lon_ls_coll, None, domain, land=True, part_size=5)
plt.plot(coastline_points.T[1], coastline_points.T[0])
plt.show()